In [1]:
import sys
sys.path.insert(0, '/Users/matthewashman/github/MasterProject2018')

# Import necessary modules. Set settings. Import data.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML

# For model building
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Miscelaneous
from IPython.display import display, clear_output
import pdb
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')

X_train = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_train.pkl')
X_validation = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_validation.pkl')
X_augmented_01 = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_augmented_01.pkl')
X_augmented_02 = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_augmented_02.pkl')
X_augmented_03 = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_augmented_03.pkl')
X_augmented_04 = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_augmented_04.pkl')
X_validation_augmented_03 = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_validation_augmented_03.pkl')
X_test = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_test.pkl')

In [2]:
# Making single correction to label
correction_idx = X_test.loc[(X_test['Patient']=='14') & (X_test['Type']=='af') & (X_test['Channel']=='CS3-4') &
                            (X_test['Coupling Interval']=='300')].index[0]

X_test.at[correction_idx, 'Label'] = 1

In [3]:
# Isolate feature matrices, target vectors and information for upsampled dataset
X_train_ = X_train.drop(['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_train = X_train['Label'].astype(int)
info_train = X_train[['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_augmented_01_ = X_augmented_01.drop(['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_augmented_01 = X_augmented_01['Label'].astype(int)
info_augmented_01 = X_augmented_01[['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_augmented_02_ = X_augmented_02.drop(['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_augmented_02 = X_augmented_02['Label'].astype(int)
info_augmented_02 = X_augmented_02[['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_augmented_03_ = X_augmented_03.drop(['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_augmented_03 = X_augmented_03['Label'].astype(int)
info_augmented_03 = X_augmented_03[['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_augmented_04_ = X_augmented_04.drop(['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_augmented_04 = X_augmented_04['Label'].astype(int)
info_augmented_04 = X_augmented_04[['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_validation_ = X_validation.drop(['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_validation = X_validation['Label'].astype(int)
info_validation = X_validation[['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_validation_augmented_03_ = X_validation_augmented_03.drop(['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_validation_augmented_03 = X_validation_augmented_03['Label'].astype(int)
info_validation_augmented_03 = X_validation_augmented_03[['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_test_ = X_test.drop(['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_test = X_test['Label'].astype(int)
info_test = X_test[['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

In [4]:
X_combined_augmented_03 = pd.concat([X_augmented_03_, X_validation_augmented_03_], ignore_index=True)
y_combined_augmented_03 = pd.concat([y_augmented_03, y_validation_augmented_03], ignore_index=True)
info_combined_augmented_03 = pd.concat([info_augmented_03, info_validation_augmented_03], ignore_index=True)

X_combined_ = pd.concat([X_train_, X_validation_], ignore_index=True)
y_combined = pd.concat([y_train, y_validation], ignore_index=True)
info_combined = pd.concat([info_train, info_validation], ignore_index=True)

## Setup Model and Estimators

In [5]:
clf = LogisticRegression(penalty='l1', C=1, random_state=1, solver='saga', multi_class='multinomial', 
                         class_weight='balanced')

### Extract Feature Matrices

In [6]:
X_aug1 = X_augmented_01_.values
X_aug2 = X_augmented_02_.values
X_aug3 = X_augmented_03_.values
X_aug4 = X_augmented_04_.values
X = X_train_.values
X_val = X_validation_.values
X_val_aug3 = X_validation_augmented_03.values
X_t = X_test_.values
X_combined_aug3 = X_combined_augmented_03.values
X_combined = X_combined_.values

y_aug1 = y_augmented_01.values
y_aug2 = y_augmented_02.values
y_aug3 = y_augmented_03.values
y_aug4 = y_augmented_04.values
y = y_train.values
y_val = y_validation.values
y_val_aug3 = y_validation_augmented_03.values
y_t = y_test.values
y_combined_aug3 = y_combined_augmented_03.values
y_combined = y_combined.values

X_info = info_train
X_val_info = info_validation
X_test_info = info_test
X_combined_info = info_combined

## Class Prediction Results

In [8]:
clf.fit(X_combined_aug3, y_combined_aug3)

predictions = clf.predict(X)
predictions_val = clf.predict(X_val)
predictions_t = clf.predict(X_t)

pp = clf.predict_proba(X)
pp_val = clf.predict_proba(X_val)
pp_t = clf.predict_proba(X_t)

print('Training Data Predictions')
cm = confusion_matrix(y, predictions)
print_cm(cm, ['Green', 'Amber','Red'])
f1 = f1_score(y, predictions, average='weighted')
print('F1 Score: ' + str(f1))

print('\nValidation Data Predictions')
cm = confusion_matrix(y_val, predictions_val)
print_cm(cm, ['Green', 'Amber','Red'])
f1 = f1_score(y_val, predictions_val, average='weighted')
print('F1 Score: ' + str(f1))

print('\Test Data Predictions')
cm = confusion_matrix(y_t, predictions_t)
print_cm(cm, ['Green', 'Amber','Red'])
f1 = f1_score(y_t, predictions_t, average='weighted')
print('F1 Score: ' + str(f1))

Training Data Predictions
     t/p  Green Amber   Red 
    Green 706.0 103.0   1.0 
    Amber  24.0  98.0  25.0 
      Red   0.0   5.0  23.0 
F1 Score: 0.8538304507646335

Validation Data Predictions
     t/p  Green Amber   Red 
    Green 244.0  21.0   1.0 
    Amber   4.0  31.0   8.0 
      Red   0.0   2.0  11.0 
F1 Score: 0.8965712727666241
\Test Data Predictions
     t/p  Green Amber   Red 
    Green 820.0 129.0   2.0 
    Amber  20.0 117.0  36.0 
      Red   0.0   2.0  31.0 
F1 Score: 0.8530997092868403


#### Visualising Predictions using LDA

In [10]:
%matplotlib qt
# Setup LDA
lda = LDA(n_components=2)
# Get LDA principle components
X_combined_lda = pd.DataFrame(data=lda.fit_transform(X_combined, y_combined), 
                     columns = ['principal component 1', 'principal component 2'])
X_t_lda = pd.DataFrame(data=lda.transform(X_t), 
                     columns = ['principal component 1', 'principal component 2'])

X_combined_lda = pd.concat([X_combined_lda, X_combined_info], axis=1)
X_t_lda = pd.concat([X_t_lda, X_test_info], axis=1)

labels = [0, 1, 2]
colors = ['g', 'orange', 'r']

fig, [ax1, ax2] = plt.subplots(ncols=2, nrows=1, figsize=(10,4), dpi=80)

for label, color in zip(labels, colors):
    idx_to_keep = (y_combined == label)
    ax1.scatter(X_combined_lda.loc[idx_to_keep, 'principal component 1'], 
                X_combined_lda.loc[idx_to_keep, 'principal component 2'],
                c = color,
                edgecolor='k',
                s = 50,
               alpha=0.1)
    
    ax2.scatter(X_combined_lda.loc[idx_to_keep, 'principal component 1'], 
                X_combined_lda.loc[idx_to_keep, 'principal component 2'],
                c = color,
                edgecolor='k',
                s = 50,
               alpha=0.1)
    
    idx_to_keep = (y_t == label)
    ax1.scatter(X_t_lda.loc[idx_to_keep, 'principal component 1'], 
                X_t_lda.loc[idx_to_keep, 'principal component 2'],
                c = color,
                edgecolor='k',
                s = 50)
    
    
    ax2.scatter(X_t_lda.loc[idx_to_keep, 'principal component 1'], 
                X_t_lda.loc[idx_to_keep, 'principal component 2'],
                c = color,
                edgecolor='k',
                s = 50,
               alpha=0.1)
    
    error_idx = ((y_t == label) & (predictions_t != y_t))
    ax2.scatter(X_t_lda.loc[error_idx, 'principal component 1'], 
                X_t_lda.loc[error_idx, 'principal component 2'],
                c=color,
                edgecolor='k',#[colors[x] for x in predictions_t[error_idx]],
                s = 50)
    
names = []
for i, row in X_t_lda.iterrows():
    row_name = row['Type'] + row['Patient'] + ' ' + row['Channel'] + ' ' + row['Coupling Interval'] + ' ' + str(row['Label'])
    names.append(row_name)
    
sc = ax2.scatter(X_t_lda['principal component 1'], X_t_lda['principal component 2'],
                 alpha=0,
                 s=50)
        
annot = ax2.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                     bbox=dict(boxstyle="round", fc="w"),
                     arrowprops=dict(arrowstyle="->"))

annot.set_visible(False)

def update_annot(ind):

    pos = sc.get_offsets()[ind["ind"][0]]
    annot.xy = pos
    text = "{}".format(" ".join([names[n] for n in ind["ind"]]))
    annot.set_text(text)


def hover(event):
    vis = annot.get_visible()
    if event.inaxes == ax2:
        cont, ind = sc.contains(event)
        if cont:
            update_annot(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                fig.canvas.draw_idle()
    
    
    
                
ax1.grid(True); ax2.grid(True)
ax1.set_axisbelow(True)
ax2.set_axisbelow(True)

fig.canvas.mpl_connect("motion_notify_event", hover)

# Remove borders
for ax in [ax1, ax2]:
    ax.spines["top"].set_alpha(0.0)    
    ax.spines["bottom"].set_alpha(0.3)
    ax.spines["right"].set_alpha(0.0)    
    ax.spines["left"].set_alpha(0.3) 
    ax.set_xlabel('Principal Component 1')
    ax.set_ylabel('Principal Component 2')
    
plt.tight_layout()
plt.show()

### Prediction Results

In [121]:
X_info['Green PP'] = pp[:,0]
X_info['Amber PP'] = pp[:,1]
X_info['Red PP'] = pp[:,2]
X_info['Predicitions'] = predictions

X_val_info['Green PP'] = pp_val[:,0]
X_val_info['Amber PP'] = pp_val[:,1]
X_val_info['Red PP'] = pp_val[:,2]
X_val_info['Predicitions'] = predictions_val

X_test_info['Green PP'] = pp_t[:,0]
X_test_info['Amber PP'] = pp_t[:,1]
X_test_info['Red PP'] = pp_t[:,2]
X_test_info['Predicitions'] = predictions_t

X_info['Weighted PP'] = pp[:,1] + 2*pp[:,2]
X_val_info['Weighted PP'] = pp_val[:,1] + 2*pp_val[:,2]
X_test_info['Weighted PP'] = pp_t[:,1] + 2*pp_t[:,2]

#### Investigate Misclassificaitons

In [124]:
%matplotlib qt

red_errors = ((y_t == 2) & (predictions_t != y_t))
red_error_idxs = [i for i, x in enumerate(red_errors) if x]
for idx in red_error_idxs:
    error_type = X_test_info.loc[idx]['Type']
    error_patient = X_test_info.loc[idx]['Patient']
    error_channel = X_test_info.loc[idx]['Channel']
    error_cs = X_test_info.loc[idx]['Coupling Interval']
    error_segment = X_test_info.loc[idx]['Data']
    error_prediction = X_test_info.loc[idx]['Weighted PP']
    
    print('Patient: ' + error_type + error_patient + '. ' + error_channel + ': ' + error_cs + '. Prediction: ' + str(error_prediction))
    
    typical_segment = X_test_info[(X_test_info['Type']==error_type) &
                                  (X_test_info['Patient']==error_patient) &
                                  (X_test_info['Channel']==error_channel)
                                 ].sort_values(by=['Coupling Interval'], ascending=False).iloc[0]['Data']
    
    fig, [ax1, ax2] = plt.subplots(nrows=2, ncols=1, figsize=(10,4), dpi=80, sharex=True, sharey=True)
    ax1.plot(typical_segment, 'k')
    ax2.plot(error_segment, 'k')
    ax1.set_title('Typical Response', fontsize=14)
    ax2.set_title('Fractionated Response', fontsize=14)
    ax2.set_xlabel('Sample (ms)', fontsize=14)
    ax2.set_ylabel(r'$\mu$V', rotation=1, fontsize=14)
    ax1.set_ylabel(r'$\mu$V', rotation=1, fontsize=14)
    
    
    # Remove borders
    for ax in [ax1, ax2]:
        ax.grid(True)
        ax.spines["top"].set_alpha(0.0)    
        ax.spines["bottom"].set_alpha(0.3)
        ax.spines["right"].set_alpha(0.0)    
        ax.spines["left"].set_alpha(0.3)  
        ax.tick_params(axis='both', which='major', labelsize=12)
        
    plt.tight_layout()
    plt.draw()
    plt.waitforbuttonpress()
    plt.close()

Patient: af14. CS1-2: 290. Prediction: 1.4903535795629554
Patient: af14. CS1-2: 280. Prediction: 1.1625159242997127


In [10]:
X_info[X_info['Type']=='af']['Patient'].unique()

array(['1', '3', '4', '6', '8', '9', '10'], dtype=object)

#### AF vs Non-AF Training Data Predictions

In [13]:
%matplotlib qt
channels = X_info['Channel'].unique()
patient_types = X_info['Type'].unique()

for patient_type in patient_types:
    patients = X_info[X_info['Type']==patient_type]['Patient'].unique()
    for patient in patients:
        X_patient = X_info[(X_info['Type']==patient_type) & (X_info['Patient']==patient)]
        
        fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(16,9))
        for channel, ax in zip(channels, axes):
            channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval']
            x = channel_cis.astype(int)
            y = X_patient[X_patient['Channel']==channel]['Weighted PP'].values
            
            ax.set_title(patient_type + patient + ': ' + channel)
            ax.set_ylabel('Weighted Prediction')
            ax.plot(x, y)
            ax.set_xlim(400, 220)
            ax.set_ylim(0, 2)
            ax.grid(True)
        
        ax.set_xlabel('Coupling Interval')
        plt.draw()
        plt.waitforbuttonpress()
        plt.close()

In [12]:
%matplotlib qt
channels = X_info['Channel'].unique()
patient_types = X_info['Type'].unique()

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(16,9))

axes[0,0].set_title('AF Patients')
axes[0,1].set_title('Non-AF Patients')

for channel, ax in zip(channels, axes[:,0]):
    ax.set_ylabel(channel)
    ax.set_xlim(400, 220)
    ax.set_ylim(0, 2)
    ax.grid(True)
    
for channel, ax in zip(channels, axes[:,1]):
    ax.set_ylabel(channel)
    ax.set_xlim(400, 220)
    ax.set_ylim(0, 2)
    ax.grid(True)
    
axes[2,0].set_xlabel('Coupling Interval')
axes[2,1].set_xlabel('Coupling Interval')

for patient_type in patient_types:
    patients = X_info[X_info['Type']==patient_type]['Patient'].unique()
    for patient in patients:
        X_patient = X_info[(X_info['Type']==patient_type) & (X_info['Patient']==patient)]
        
        if patient_type == 'af':
            for channel, ax in zip(channels, axes[:,0]):
                channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval']
                x = channel_cis.astype(int)
                y = X_patient[X_patient['Channel']==channel]['Weighted PP'].values       
                ax.plot(x, y, 'C2')
                
        else:
            for channel, ax in zip(channels, axes[:,1]):
                channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval']
                x = channel_cis.astype(int)
                y = X_patient[X_patient['Channel']==channel]['Weighted PP'].values
                ax.plot(x, y, 'C3')

plt.show()

#### AF vs Non-AF Validation Data Predictions

In [13]:
%matplotlib qt
channels = X_val_info['Channel'].unique()
patient_types = X_val_info['Type'].unique()

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(16,9))

axes[0,0].set_title('AF Patients')
axes[0,1].set_title('Non-AF Patients')

for channel, ax in zip(channels, axes[:,0]):
    ax.set_ylabel(channel)
    ax.set_xlim(400, 220)
    ax.set_ylim(0, 2)
    ax.grid(True)
    
for channel, ax in zip(channels, axes[:,1]):
    ax.set_ylabel(channel)
    ax.set_xlim(400, 220)
    ax.set_ylim(0, 2)
    ax.grid(True)
    
axes[2,0].set_xlabel('Coupling Interval')
axes[2,1].set_xlabel('Coupling Interval')

for patient_type in patient_types:
    patients = X_val_info[X_val_info['Type']==patient_type]['Patient'].unique()
    for patient in patients:
        X_patient = X_val_info[(X_val_info['Type']==patient_type) & (X_val_info['Patient']==patient)]
        
        if patient_type == 'af':
            for channel, ax in zip(channels, axes[:,0]):
                channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval']
                x = channel_cis.astype(int)
                y = X_patient[X_patient['Channel']==channel]['Weighted PP'].values       
                ax.plot(x, y, 'C2')
                
        else:
            for channel, ax in zip(channels, axes[:,1]):
                channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval']
                x = channel_cis.astype(int)
                y = X_patient[X_patient['Channel']==channel]['Weighted PP'].values
                ax.plot(x, y, 'C3')

plt.show()

#### AF vs Non-AF Test Predictions

In [12]:
%matplotlib qt
channels = X_test_info['Channel'].unique()
patient_types = X_test_info['Type'].unique()

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(16,9))

axes[0,0].set_title('AF Patients')
axes[0,1].set_title('Non-AF Patients')

for channel, ax in zip(channels, axes[:,0]):
    ax.set_ylabel(channel)
    ax.set_xlim(400, 220)
    ax.set_ylim(0, 2)
    ax.grid(True)
    
for channel, ax in zip(channels, axes[:,1]):
    ax.set_ylabel(channel)
    ax.set_xlim(400, 220)
    ax.set_ylim(0, 2)
    ax.grid(True)
    
axes[2,0].set_xlabel('Coupling Interval')
axes[2,1].set_xlabel('Coupling Interval')

for patient_type in patient_types:
    patients = X_test_info[X_test_info['Type']==patient_type]['Patient'].unique()
    for patient in patients:
        X_patient = X_test_info[(X_test_info['Type']==patient_type) & (X_test_info['Patient']==patient)]
        
        if patient_type == 'af':
            for channel, ax in zip(channels, axes[:,0]):
                channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval'].values
                x = channel_cis.astype(int)
                y = X_patient[X_patient['Channel']==channel]['Weighted PP'].values       
                ax.plot(x, y, 'C2')
                patient_name = patient_type + patient
                ax.annotate(patient_name, xy=[x[-1],y[-1]])
                
        else:
            for channel, ax in zip(channels, axes[:,1]):
                channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval'].values
                x = channel_cis.astype(int)
                y = X_patient[X_patient['Channel']==channel]['Weighted PP'].values
                ax.plot(x, y, 'C3')
                patient_name = patient_type + patient
                ax.annotate(patient_name, xy=[x[-1],y[-1]])

plt.show()

In [51]:
X_test_info.shape

(213, 20)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

best_f1 = 0
for num_features in range(1, len(X_train.columns)):
    selector = RFE(estimator, num_features, step=1)
    selector = selector.fit(X_aug.values, y_aug)
    print('\nNum Features: ' + str(num_features))
    features = X_train_aug.columns[selector.support_]
    print(features)
    
    print('~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('Test score:')
    clf.fit(X_aug[features].values, y_aug)
    predictions = clf.predict(X_val_aug[features].values)
    cm = confusion_matrix(y_val, predictions)
    print_cm(cm, ['Green', 'Amber','Red'])
    f1 = f1_score(y_val, predictions, average='weighted')
    print('F1 Score: ' + str(f1))

    if f1 > best_f1:
        best_features = features
        best_f1 = f1
        
print(best_features)
print(best_f1)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

best_f1 = 0
for num_features in range(1, len(X_train_aug.columns)):
    selector = RFE(estimator, num_features, step=1)
    selector = selector.fit(X_train_aug.values, y_train_aug.values)
    print('\nNum Features: ' + str(num_features))
    features = X_train_aug.columns[selector.support_]
    print(features)
    
    print('~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('Test score:')
    clf.fit(X_train_aug[features].values, y_train_aug.values)
    predictions = clf.predict(X_validation[features].values)
    cm = confusion_matrix(y_validation.values, predictions)
    print_cm(cm, ['Green', 'Amber','Red'])
    f1 = f1_score(y_validation.values, predictions, average='weighted')
    print('F1 Score: ' + str(f1))

    if f1 > best_f1:
        best_features = features
        best_f1 = f1
        
print(best_features)
print(best_f1)

In [7]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    
    # Begin CHANGES
    fst_empty_cell = (columnwidth-3)//2 * " " + "t/p" + (columnwidth-3)//2 * " "
    
    if len(fst_empty_cell) < len(empty_cell):
        fst_empty_cell = " " * (len(empty_cell) - len(fst_empty_cell)) + fst_empty_cell
    # Print header
    print("    " + fst_empty_cell, end=" ")
    # End CHANGES
    
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
        
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()